# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [1]:
# Import initial libraries
# Had to install citipy in Folder Module terminal
import numpy as np
import pandas as pd
from citipy import citipy

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list
# Unpacks the zip object into a list
coordinates = list(lat_lngs)
coordinates

[(3.982717375633669, -67.10907121345704),
 (76.55566884949425, 41.1181825751934),
 (-8.4629740595243, -146.70967563545818),
 (-25.246334700808873, -65.97078365587903),
 (80.09484815802836, -132.8238700009963),
 (53.29598946743593, -115.85694615996054),
 (80.64116477491456, -31.200351685926336),
 (-39.841839035410665, 165.8198469712783),
 (14.763436408672234, 94.69706519413398),
 (50.995805343568065, -42.348004840850365),
 (20.202434280590538, -7.161839616402261),
 (-47.54884615420227, -39.85226821956513),
 (22.934173387506263, 179.0040319386501),
 (-47.7101697042971, -100.48824502345853),
 (-68.68923798121395, 51.389548710997644),
 (-71.03257155304792, 40.89736505212139),
 (21.00054163183347, 46.64486954140153),
 (-39.31724108959091, 103.17146739481944),
 (70.56920093986602, 75.86441548704042),
 (2.136373204636925, 10.76455748616533),
 (-0.7896130577977232, -102.08773763460613),
 (51.03031793812511, 124.23773314115613),
 (-4.109057280775957, -88.01313427329897),
 (83.13700841769602, 54

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [4]:
# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

590

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [5]:
# Import the requests library
import requests

# Import the time library
import time

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

In [6]:
# Assemble the OpenWeatherMap's API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=d5aa8b2e67e7758e64fb7c1bbaf1f060


## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [7]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description
        lat = city_weather["coord"]["lat"]
        lng = city_weather["coord"]["lon"]
        max_temp = city_weather["main"]["temp_max"]
        humidity = city_weather["main"]["humidity"]
        clouds = city_weather["clouds"]["all"]
        wind = city_weather["wind"]["speed"]
        country = city_weather["sys"]["country"]
        weather_description = city_weather["weather"][0]["description"]
        # convert the date to ISO - Module 6.2.5
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            
        # Append the city information into the city_data list
        city_data.append({"City": city.title(),
                          "Lat": lat,
                          "Lng": lng,
                          "Max Temp": max_temp,
                          "Humidity": humidity,
                          "Cloudiness": clouds,
                          "Wind Speed": wind,
                          "Country": country,
                          "Current Description": weather_description })
    
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | inirida
Processing Record 2 of Set 1 | polyarnyy
Processing Record 3 of Set 1 | taiohae
City not found. Skipping...
Processing Record 4 of Set 1 | chicoana
Processing Record 5 of Set 1 | whitehorse
Processing Record 6 of Set 1 | edson
Processing Record 7 of Set 1 | isafjordur
Processing Record 8 of Set 1 | queenstown
Processing Record 9 of Set 1 | bogale
Processing Record 10 of Set 1 | qaqortoq
Processing Record 11 of Set 1 | nema
Processing Record 12 of Set 1 | grytviken
Processing Record 13 of Set 1 | utrik
Processing Record 14 of Set 1 | chonchi
Processing Record 15 of Set 1 | port-aux-francais
Processing Record 16 of Set 1 | as sulayyil
Processing Record 17 of Set 1 | margaret river
Processing Record 18 of Set 1 | tazovsky
Processing Record 19 of Set 1 | mikomeseng
Processing Record 20 of Set 1 | puerto ayora
Processing Record 21 of Set 1 | jiagedaqi
Processing Record 22 of Set 1 | puerto baq

In [8]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

566

## Add the weather data to a new DataFrame.

In [9]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)


In [10]:
# Display sample data
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Inirida,3.8653,-67.9239,77.49,88,48,2.06,CO,scattered clouds
1,Polyarnyy,69.1989,33.4478,37.89,99,100,6.87,RU,overcast clouds
2,Chicoana,-25.1009,-65.5331,61.05,74,6,3.42,AR,clear sky
3,Whitehorse,60.7161,-135.0538,57.97,41,100,19.57,CA,overcast clouds
4,Edson,53.5835,-116.4356,71.74,28,0,6.91,CA,clear sky
5,Isafjordur,66.0755,-23.1240,41.29,93,97,11.32,IS,overcast clouds
6,Queenstown,-31.8976,26.8753,49.32,90,100,2.15,ZA,light rain
7,Bogale,16.2833,95.4000,79.27,89,100,8.61,MM,overcast clouds
8,Qaqortoq,60.7167,-46.0333,31.82,87,84,8.63,GL,broken clouds
9,Nema,57.5067,50.5011,50.40,61,35,4.43,RU,scattered clouds


In [11]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Current Description'],
      dtype='object')

In [12]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
column_reorder=['City','Country','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed', 'Current Description']

# Recreate the DataFrame by using the new column order
city_data_df = city_data_df[column_reorder]

# Display sample data
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Inirida,CO,3.8653,-67.9239,77.49,88,48,2.06,scattered clouds
1,Polyarnyy,RU,69.1989,33.4478,37.89,99,100,6.87,overcast clouds
2,Chicoana,AR,-25.1009,-65.5331,61.05,74,6,3.42,clear sky
3,Whitehorse,CA,60.7161,-135.0538,57.97,41,100,19.57,overcast clouds
4,Edson,CA,53.5835,-116.4356,71.74,28,0,6.91,clear sky
...,...,...,...,...,...,...,...,...,...
561,Bocaiuva,BR,-17.1078,-43.8150,80.31,58,72,3.13,broken clouds
562,Pimampiro,EC,0.4000,-77.9667,64.60,84,100,2.21,light rain
563,Gori,GE,41.9842,44.1158,56.14,97,100,9.55,overcast clouds
564,Laguna,US,38.4210,-121.4238,87.08,34,0,8.05,clear sky


In [13]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [14]:
# Set the output file name
output_file = "csv_Weather/WeatherPy_Database.csv"

# Export the city_data DataFrame into a CSV file
city_data_df.to_csv(output_file, index_label="City_ID")